In [2]:
import pickle
import re
import os
import json
import numpy as np
import pandas as pd

path = "/home/mlb/res/stock/twse/json/"
regex = re.compile(r"[12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])")


In [53]:
def check_null(info):
    for key, value in info.items(): #avoid null value
        if value=='NULL':
            return False
    return True

def loader(number,filename):
    result = []
    for name in os.listdir(path):
        if(regex.match(name)):
            with open(os.path.join(path,name)) as f:
                data = json.load(f)
                
                try:
                    s=data[number]
                    if(check_null(s)):
                        s['date'] = ''.join(name[0:10].split('-'))
                        result.append(s)
                except KeyError:
                    print(f'Can not find the information of {number} in file {name}')
    if len(result):
        with open(filename+".pickle",'wb') as output:
            df = pd.DataFrame(result).sort_values(by=['date'])
            df[['adj_close','close','high','low','open']]=df[['adj_close','close','high','low','open']].astype('float64')
            df[['volume','date']]=df[['volume','date']].astype('int')
            df=df.reset_index(drop=True)
            pickle.dump(df,output)
    else:
        print("Empty result can not generate output file")
        


In [55]:
def reader(filename):
    with open(filename+".pickle",'rb') as f:
        df = pickle.load(f)
        print(df)
        #print(df.iloc[0]['low'])




In [56]:
loader('0050',"0050")

      adj_close   close    high     low    open   volume      date
0         48.30   48.37   48.50   48.26   48.50  5264000  20040211
1         48.54   48.59   49.26   48.40   48.50  8114000  20040212
2         49.41   49.46   49.47   48.60   48.61  6138000  20040213
3         49.17   49.20   49.46   49.04   49.46  4154000  20040216
4         49.40   49.50   49.50   49.16   49.21  4645000  20040217
...         ...     ...     ...     ...     ...      ...       ...
4109     102.95  103.00  103.45  102.60  102.75  3545485  20200930
4110     103.05  103.05  104.05  103.00  103.45  5912599  20201005
4111     104.20  104.25  104.35  103.85  104.00  6875260  20201006
4112      90.10   90.10   90.60   89.90   89.90  9458164  20201007
4113      90.10   90.10   90.60   89.90   89.90  9458164  20201008

[4114 rows x 7 columns]


### Idea:
* pick the data of last 7 days as input

In [120]:
with open("0050.pickle",'rb') as f:
    df = pickle.load(f)
    
    df.info()
    data_x=[]
    data_y=[]
    print(len(df))
    for i in range(7,len(df)-1):
        batch=df.iloc[i-7:i]
        data_x.append(list(batch['high'].values[:]))
        data_y.append(df['high'][i])
        
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4114 entries, 0 to 4113
Data columns (total 7 columns):
adj_close    4114 non-null float64
close        4114 non-null float64
high         4114 non-null float64
low          4114 non-null float64
open         4114 non-null float64
volume       4114 non-null int64
date         4114 non-null int64
dtypes: float64(5), int64(2)
memory usage: 225.1 KB
4114


In [115]:
data_y

[50.05,
 50.6,
 52.0,
 52.9,
 52.95,
 53.15,
 53.7,
 52.9,
 52.0,
 52.1,
 51.45,
 50.8,
 51.0,
 49.38,
 49.7,
 50.8,
 51.1,
 47.43,
 47.17,
 47.3,
 47.25,
 47.3,
 49.1,
 49.54,
 49.5,
 49.3,
 49.2,
 50.5,
 50.45,
 49.98,
 50.1,
 49.9,
 51.05,
 51.1,
 51.4,
 51.6,
 51.35,
 51.3,
 51.5,
 51.35,
 52.2,
 51.15,
 51.15,
 50.9,
 50.55,
 49.7,
 48.5,
 47.1,
 47.0,
 47.1,
 45.7,
 45.9,
 45.15,
 45.1,
 45.75,
 45.35,
 45.42,
 43.51,
 42.55,
 44.61,
 45.35,
 45.59,
 45.69,
 45.6,
 46.49,
 46.1,
 46.3,
 46.31,
 45.35,
 44.9,
 45.15,
 43.64,
 45.25,
 46.0,
 45.6,
 45.15,
 44.5,
 43.82,
 43.0,
 43.5,
 43.8,
 43.29,
 43.7,
 44.3,
 44.5,
 44.7,
 44.5,
 43.83,
 44.85,
 45.05,
 44.2,
 43.68,
 43.8,
 43.99,
 44.2,
 44.4,
 44.52,
 43.98,
 43.85,
 43.3,
 42.73,
 42.56,
 42.36,
 42.18,
 41.8,
 42.15,
 41.24,
 42.0,
 42.2,
 41.72,
 42.1,
 41.8,
 41.85,
 41.6,
 42.8,
 42.79,
 42.24,
 42.24,
 42.32,
 41.89,
 42.08,
 41.89,
 41.9,
 42.88,
 44.22,
 44.49,
 44.59,
 46.0,
 45.95,
 45.7,
 45.95,
 45.85,
 45.84,
 4

In [119]:
data_x

[[48.5, 49.26, 49.47, 49.46, 49.5, 49.7, 50.1],
 [49.26, 49.47, 49.46, 49.5, 49.7, 50.1, 50.05],
 [49.47, 49.46, 49.5, 49.7, 50.1, 50.05, 50.0],
 [49.46, 49.5, 49.7, 50.1, 50.05, 50.0, 49.62],
 [49.5, 49.7, 50.1, 50.05, 50.0, 49.62, 49.62],
 [49.7, 50.1, 50.05, 50.0, 49.62, 49.62, 50.05],
 [50.1, 50.05, 50.0, 49.62, 49.62, 50.05, 50.6],
 [50.05, 50.0, 49.62, 49.62, 50.05, 50.6, 52.0],
 [50.0, 49.62, 49.62, 50.05, 50.6, 52.0, 52.9],
 [49.62, 49.62, 50.05, 50.6, 52.0, 52.9, 52.95],
 [49.62, 50.05, 50.6, 52.0, 52.9, 52.95, 53.15],
 [50.05, 50.6, 52.0, 52.9, 52.95, 53.15, 53.7],
 [50.6, 52.0, 52.9, 52.95, 53.15, 53.7, 52.9],
 [52.0, 52.9, 52.95, 53.15, 53.7, 52.9, 52.0],
 [52.9, 52.95, 53.15, 53.7, 52.9, 52.0, 52.1],
 [52.95, 53.15, 53.7, 52.9, 52.0, 52.1, 51.45],
 [53.15, 53.7, 52.9, 52.0, 52.1, 51.45, 50.8],
 [53.7, 52.9, 52.0, 52.1, 51.45, 50.8, 51.0],
 [52.9, 52.0, 52.1, 51.45, 50.8, 51.0, 49.38],
 [52.0, 52.1, 51.45, 50.8, 51.0, 49.38, 49.7],
 [52.1, 51.45, 50.8, 51.0, 49.38, 49.7, 50

In [123]:
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

data_X,data_Y=shuffle(data_x,data_y, random_state=1)

train_x, test_x,train_y,test_y = train_test_split(data_X, data_Y, test_size=0.33, random_state=2)
clf = Pipeline([
    ('scl', StandardScaler()),
    ('pca', PCA(n_components=5)),
    #('clf',SVR())
    ('clf', RandomForestRegressor(n_estimators=50))
])
clf.fit(train_x, train_y)
print(clf.predict([[102.3,103.25,103.45,104.05,104.35,105,106.35]]))
print(clf.score(test_x, test_y))



[104.368]
0.9958954548787964
